# WISE + NEOWISE-R Level 1b 

From single-epoch images from WISE and NEOWISE-R, retrieve image footprints over the entire sky.

In [2]:
import pyvo as vo
import numpy as np
import astropy.coordinates as coord

After some investigation on IRSA's API pages, we want a TAP query to wise.neowiser_merge_p1bm_frm

In [5]:
tap_services = vo.regsearch(servicetype='table', keywords=['irsa'])

In [6]:
tap_services

<Table masked=True length=1>
       ivoid        cap_index intf_index ... rights_uri      harvested_from     
                                         ...                                    
       object         int16     int16    ...   object            object         
------------------- --------- ---------- ... ---------- ------------------------
ivo://irsa.ipac/tap         1          1 ...            ivo://irsa.ipac/registry

There is one IRSA TAP service. Select it.

In [7]:
service = tap_services[0]

Select only ten results from this table.

In [8]:
frm_result = service.search("SELECT TOP 10 * " +
                        "FROM wise.neowiser_merge_p1bm_frm ")

In [9]:
frm_result

<Table masked=True length=10>
 band naxis naxis1 naxis2 ...          z           spt_ind  image_set
int32 int32 int32  int32  ...       float64         int32     int32  
----- ----- ------ ------ ... ------------------- --------- ---------
    4     2    508    508 ...  0.5961361225753120 212202012         4
    2     2   1016   1016 ...  0.5898352201999200 212202001         6
    3     2   1016   1016 ...  0.6083831383073640 212202331         4
    3     2   1016   1016 ...  0.6226102703766390 212203211         4
    1     2   1016   1016 ...  0.6186221279416710 212203321         6
    1     2   1016   1016 ...  0.6177134827181080 212203321         4
    1     2   1016   1016 ...  0.6193563765740000 220103121         6
    1     2   1016   1016 ...  0.6231831235160790 220103131         6
    1     2   1016   1016 ...  0.6013070208720001 212202011         7
    1     2   1016   1016 ...  0.6062238341829950 212202321         4

What columns are available?

In [10]:
frm_result.to_table().colnames

['band',
 'naxis',
 'naxis1',
 'naxis2',
 'wrelease',
 'crpix1',
 'crpix2',
 'crval1',
 'crval2',
 'ctype1',
 'ctype2',
 'equinox',
 'bunit',
 'elon',
 'elat',
 'glon',
 'glat',
 'ra1',
 'dec1',
 'ra2',
 'dec2',
 'ra3',
 'dec3',
 'ra4',
 'dec4',
 'magzp',
 'magzpunc',
 'modeint',
 'scan_id',
 'scangrp',
 'frame_num',
 'l0file',
 'date_obs',
 'mjd_obs',
 'icaldir',
 'dtanneal',
 'utanneal',
 'unixt',
 'ephemt',
 'exptime',
 'tsamp',
 'wcdelt1',
 'wcdelt2',
 'crder1',
 'crder2',
 'csdradec',
 'pxscal1',
 'pxscal2',
 'uncrts1',
 'uncrts2',
 'wcrota2',
 'pa',
 'uncrtpa',
 'skew',
 'cd1_1',
 'cd1_2',
 'cd2_1',
 'cd2_2',
 'debgain',
 'febgain',
 'moon_sep',
 'saa_sep',
 'qa_status',
 'qual_frame',
 'qc_fact',
 'qi_fact',
 'qn_fact',
 'qa_fact',
 'q1_fact',
 'q2_fact',
 'q3_fact',
 'q4_fact',
 'q5_fact',
 'qual_scan',
 'qs1_fact',
 'qs5_fact',
 'qp_fact',
 'qs_fact',
 'q6_fact',
 'q7_fact',
 'q8_fact',
 'q9_fact',
 'q10_fact',
 'date_imgprep',
 'load_id',
 'cntr',
 'x',
 'y',
 'z',
 'spt_ind'

How many rows are in this table?

In [12]:
%%time
cnt_result = service.search("SELECT count(elon) " +
                        "FROM wise.neowiser_merge_p1bm_frm ")

CPU times: user 19.1 ms, sys: 4.43 ms, total: 23.5 ms
Wall time: 1min 40s


In [13]:
cnt_result

<Table masked=True length=1>
 count  
 int64  
--------
34549715

In [25]:
job = service.service.submit_job("SELECT TOP 1000 mjd_obs,elon,elat,ra1,dec1,ra2,dec2," +
                                 "ra3,dec3,ra4,dec4,crval1,crval2 "
                        "FROM wise.neowiser_merge_p1bm_frm ")

In [26]:
job.run()

In [ ]:
from time import sleep
sleep(300)

In [27]:
print(job.phase)

COMPLETED


In [30]:
top1000 = job.fetch_result()

In [33]:
mytab = top1000.to_table()

In [37]:
myarray = mytab.as_array()

In [38]:
myarray.nbytes

104000

In [39]:
cnt_result.to_table()['count'][0]

34549715

In [40]:
num_bytes_all = myarray.nbytes * cnt_result.to_table()['count'][0] / 1000

In [41]:
num_bytes_all

3593170360.0

In [43]:
print('Number of GB to query everything is ', num_bytes_all * 1024**-3)

Number of GB to query everything is  3.346400670707226


In [44]:
#full_job = service.service.submit_job("SELECT mjd_obs,elon,elat,ra1,dec1,ra2,dec2," +
#                                 "ra3,dec3,ra4,dec4,crval1,crval2 "
#                        "FROM wise.neowiser_merge_p1bm_frm ")

In [45]:
#full_job.run()

In [54]:
#print(full_job.phase)

EXECUTING


In [55]:
#print(full_job.url)

https://irsa.ipac.caltech.edu/TAP/async/10707127


In [52]:
#%pdb on
#full_job.wait()

Automatic pdb calling has been turned ON


AttributeError: 'LooseVersion' object has no attribute 'version'

> /Users/shupe/miniconda3/envs/irsa-notebooks/lib/python3.7/distutils/version.py(335)_cmp()
    333             other = LooseVersion(other)
    334 
--> 335         if self.version == other.version:
    336             return 0
    337         if self.version < other.version:



ipdb>  p self.version


*** AttributeError: 'LooseVersion' object has no attribute 'version'


ipdb>  p other


LooseVersion ('1.1')


ipdb>  q


In [53]:
#job.version

AttributeError: 'AsyncTAPJob' object has no attribute 'version'

> <ipython-input-53-ab936c7e730c>(1)<module>()
----> 1 job.version



ipdb>  q
